In [1]:
import pandas as pd

# gl = pd.read_csv('./data/game_logs.csv')
gl = pd.read_csv('./data/game_logs.csv', low_memory=False) # 来避免分块读取文件时的警告
gl.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


In [2]:
gl.shape

(171907, 161)

In [3]:
gl.info(memory_usage='deep') # 查看内存使用情况

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 778.5 MB


In [4]:
# 查看每列的数据类型 占用内存大小
for dtype in ['float64','object','int64']:
    selected_dtype = gl.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean() # deep=True 会考虑对象的内存使用情况
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float64 columns: 1.29 MB
Average memory usage for object columns: 8.48 MB
Average memory usage for int64 columns: 1.12 MB


In [5]:
import numpy as np
int_types = ["uint8", "int8", "int16","int32","int64"]
#unit8 无符号8位整数 int8 有符号8位整数 int16 有符号16位整数 int32 有符号32位整数 int64 有符号64位整数  
for it in int_types:
    print(np.iinfo(it)) # 查看各个整数类型的取值范围

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for int32
---------------------------------------------------------------
min = -2147483648
max = 2147483647
---------------------------------------------------------------

Machine parameters for int64
---------------------------------------------------------------
min = -9223372036854775808
max = 9223372036854775807
---------------------------------------------------------------



In [6]:
# 通过查看整数类型的取值范围，可以选择合适的整数类型来存储数据 从而减少内存使用
# 例如，如果一个列的最大值是255，那么可以使用uint8类型来存储数据
# 通过downcast参数可以将数据转换为合适的整数类型

def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

gl_int = gl.select_dtypes(include=['int64'])
# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_numeric.html
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned') # 将数据转换为无符号整数类型

print(mem_usage(gl_int))
print(mem_usage(converted_int))

7.87 MB
1.48 MB


In [7]:
gl_float = gl.select_dtypes(include=['float64'])
converted_float = gl_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(gl_float))
print(mem_usage(converted_float))

100.99 MB
50.49 MB


In [8]:
optimized_gl = gl.copy()

optimized_gl[converted_int.columns] = converted_int
optimized_gl[converted_float.columns] = converted_float

print(mem_usage(gl))
print(mem_usage(optimized_gl))

778.55 MB
721.66 MB


In [9]:
# object类型的数据可以通过pandas的category类型来存储，从而减少内存使用
gl_obj = gl.select_dtypes(include=['object']).copy()
gl_obj.describe()

,day_of_week,v_name,v_league,h_name,h_league,day_night,completion,forefeit,protest,park_id,...,h_player_6_id,h_player_6_name,h_player_7_id,h_player_7_name,h_player_8_id,h_player_8_name,h_player_9_id,h_player_9_name,additional_info,acquisition_info
count,171907,171907,171907,171907,171907,140150,116,145,180,171907,...,140838,140838,140838,140838,140838,140838,140838,140838,1456,140841
unique,7,148,7,148,7,2,116,3,5,245,...,4774,4720,5253,5197,4760,4710,5193,5142,332,1
top,Sat,CHN,NL,CHN,NL,D,"19200904,,0,6,36",H,V,STL07,...,grimc101,Charlie Grimm,grimc101,Charlie Grimm,lopea102,Al Lopez,spahw101,Warren Spahn,HTBF,Y
freq,28891,8870,88866,9024,88867,82724,1,69,90,7022,...,427,427,491,491,676,676,339,339,1112,140841


In [10]:
dow = gl_obj.day_of_week
print(dow.head())

dow_cat = dow.astype('category')
print(dow_cat.head())

0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: object
0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: category
Categories (7, object): ['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed']


In [11]:
dow_cat.head().cat.codes # 查看category类型的编码 
# .cat 用来访问category类型的属性和方法 .codes 查看category类型的编码

0    4
1    0
2    2
3    1
4    5
dtype: int8

In [12]:
print(mem_usage(dow))
print(mem_usage(dow_cat))

8.53 MB
0.16 MB


In [13]:
# 将所有的object类型的数据转换为category类型
converted_obj = pd.DataFrame()

# 对每一列进行处理 如果某一列的唯一值占总数的50%以下，则转换为category类型 否则不转换
# 如果唯一值的数量占总数的比例较低（例如低于50%），则转换为 category 类型可以显著减少内存使用。
# 然而，如果唯一值的数量占总数的比例较高（例如超过50%），
# 转换为 category 类型可能不会带来显著的内存节省，甚至可能增加复杂性。
# 因此，通常在唯一值占比较低时才进行转换。

for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = gl_obj[col]

In [14]:
print(mem_usage(gl_obj))
print(mem_usage(converted_obj))

669.69 MB
47.70 MB


In [15]:
# 将转换后的数据存储到原始数据中
optimized_gl[converted_obj.columns] = converted_obj

# 查看转换后的数据内存使用情况
mem_usage(optimized_gl)

'99.67 MB'

In [16]:
date = optimized_gl.date
print(mem_usage(date))
date.head()

0.66 MB


0    18710504
1    18710505
2    18710506
3    18710508
4    18710509
Name: date, dtype: uint32

In [17]:
# 标准日期格式 
optimized_gl['date'] = pd.to_datetime(date, format='%Y%m%d')
print (mem_usage(optimized_gl['date']))

1.31 MB


In [18]:
optimized_gl['date'][:5]

0   1871-05-04
1   1871-05-05
2   1871-05-06
3   1871-05-08
4   1871-05-09
Name: date, dtype: datetime64[ns]

In [19]:
# 将日期数据转换为datetime类型，占用较大内存
optimized_gl['date'] = pd.to_datetime(date, format='%Y%m%d')

print(mem_usage(optimized_gl))

100.33 MB


In [20]:
optimized_gl.date.head()

0   1871-05-04
1   1871-05-05
2   1871-05-06
3   1871-05-08
4   1871-05-09
Name: date, dtype: datetime64[ns]

In [24]:
# 日期数据可以转换为unit32类型，从而减少内存使用
optimized_gl['date'] = optimized_gl['date'].astype('uint32')
print(optimized_gl['date'].head())

0    18710504
1    18710505
2    18710506
3    18710508
4    18710509
Name: date, dtype: uint32


In [25]:
print(mem_usage(optimized_gl))

99.67 MB
